In [8]:
import json
import pandas as pd
from tqdm import tqdm

### Extract Headers
Identify which section headers might be relevant to our experimental design

In [9]:
def extract_keys(data, one_giant_list):

    for entry in data['results']:
        for key in list(entry.keys()):
            one_giant_list.append(key)

    return(one_giant_list)


In [17]:
file_numbers = ['0001','0002','0003','0004','0005','0006','0007','0008','0009','0010','0011','0012']
keys_list = []

for number in tqdm(file_numbers):
    f = open(f'../data/drug-label-{number}-of-0012.json','r')
    data = json.load(f)
    keys_list = extract_keys(data, keys_list)

keys_df = pd.DataFrame({'keys' : keys_list})


100%|██████████| 12/12 [00:12<00:00,  1.03s/it]


In [18]:
print(len(keys_df['keys'].value_counts()))
keys_df['keys'].value_counts()[0:50]

keys_df['keys'].value_counts().reset_index().to_csv('keys.csv')

159


### Extract Data
Using the openFDA json set, extract the relevant text fields from each application, where they exist. Save this data to a growing dataframe and then save the dataframe

In [12]:
HEADERS = ['brand_name',
           'application_number',
            'adverse_reactions',
            'clinical_studies',
            'indications_and_usage',
            'contraindications',
            'warnings_and_cautions',
            'warnings',
            'precautions',
            'pharmacokinetics',
            'purpose',
            'clinical_pharmacology',
            'active_ingredient',
            'stop_use',
            'boxed_warning',
            'pharmacodynamics',
            'pharmacogenomics'
            ]


In [13]:
def extract_data_point(data):
    tdf = pd.DataFrame(columns=HEADERS)
    for entry in data['results']:
        build_dict = {}
        for header in HEADERS:
            if header == 'brand_name': #TODO: Handle broken brand names
                try:
                    build_dict[header] = entry['openfda'][header][0]
                except:
                    build_dict[header] = None
            elif header == 'application_number': #TODO: Handle broken application numbers
                try:
                    build_dict[header] = entry['openfda'][header][0]
                except:
                    build_dict[header] = None

            else:
                try:
                    build_dict[header] = entry[header][0]
                except:
                    build_dict[header] = None

        tdf = pd.concat([tdf,pd.DataFrame.from_dict([build_dict])],axis=0).reset_index(drop=True)

    return(tdf)

# "openfda": {
        # "application_number": [

In [19]:
file_numbers = ['0001','0002','0003','0004','0005','0006','0007','0008','0009','0010','0011','0012']
fda_df = pd.DataFrame(columns=HEADERS)

for number in tqdm(file_numbers):
    f = open(f'../data/drug-label-{number}-of-0012.json','r')
    data = json.load(f)
    fda_df = pd.concat([fda_df,extract_data_point(data)],axis=0).reset_index(drop=True)

fda_df

100%|██████████| 12/12 [07:50<00:00, 39.17s/it]


,brand_name,application_number,adverse_reactions,clinical_studies,indications_and_usage,contraindications,warnings_and_cautions,warnings,precautions,pharmacokinetics,purpose,clinical_pharmacology,active_ingredient,stop_use,boxed_warning,pharmacodynamics,pharmacogenomics
0,ACETAMINOPHEN AND CODEINE PHOSPHATE,ANDA040419,ADVERSE REACTIONS The following serious advers...,None,INDICATIONS AND USAGE Acetaminophen and codein...,CONTRAINDICATIONS Acetaminophen and codeine ph...,None,"WARNINGS Addiction, Abuse, and Misuse Acetamin...",PRECAUTIONS Risks of Driving and Operating Mac...,None,None,CLINICAL PHARMACOLOGY Mechanism of Action Code...,None,None,WARNING: SERIOUS AND LIFE-THREATENING RISKS FR...,None,None
1,Magnesium Sulfate in Water,ANDA209642,ADVERSE REACTIONS The adverse effects of paren...,None,INDICATIONS AND USAGE Magnesium Sulfate in Wat...,CONTRAINDICATIONS Intravenous magnesium should...,None,WARNINGS FETAL HARM: Continuous administration...,PRECAUTIONS Because magnesium is removed from ...,Pharmacokinetics Absorption: Intravenously adm...,None,CLINICAL PHARMACOLOGY Magnesium (Mg ++ ) is an...,None,None,None,None,None
2,omeprazole,ANDA216096,None,None,Use • treats frequent heartburn (occurs 2 or m...,None,None,Warnings Allergy alert: • do not use if you ar...,None,None,Purpose Acid reducer,None,Active ingredient (in each capsule) Omeprazole...,Stop use and ask a doctor if: • your heartburn...,None,None,None
3,None,None,None,None,"Uses When using this product, avoid contact wi...",None,None,Warnings For external use only . When using th...,None,None,Purpose Antimicrobial,None,Active ingredient Benzalkonium Chloride 0.1% P...,Stop use and ask a doctor if irritation or red...,None,None,None
4,Zenpep,BLA022210,6 ADVERSE REACTIONS The following serious or o...,14 CLINICAL STUDIES Adult and Pediatric Patien...,1 INDICATIONS AND USAGE ZENPEP ® is indicated ...,4 CONTRAINDICATIONS None None ( 4 ),5 WARNINGS AND PRECAUTIONS • Fibrosing Colonop...,None,None,12.3 Pharmacokinetics Following oral administr...,None,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,None,None,None,12.2 Pharmacodynamics For patients consuming a...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223999,Dacarbazine,ANDA075259,"ADVERSE REACTIONS Symptoms of anorexia, nausea...",None,INDICATIONS AND USAGE Dacarbazine for Injectio...,CONTRAINDICATIONS Dacarbazine for injection is...,None,WARNINGS Hemopoietic depression is the most co...,PRECAUTIONS Hospitalization is not always nece...,None,None,CLINICAL PHARMACOLOGY After intravenous admini...,None,None,WARNING It is recommended that dacarbazine for...,None,None
224000,Articadent,NDA020971,6 ADVERSE REACTIONS Reactions to articaine are...,6.1 Clinical Studies Experience Because clinic...,1 INDICATIONS AND USAGE ARTICADENT is indicate...,4 CONTRAINDICATIONS ARTICADENT is contraindica...,5 WARNINGS AND PRECAUTIONS Accidental Intravas...,None,None,12.3 Pharmacokinetics Absorption: Following de...,None,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,None,None,None,"12.2 Pharmacodynamics Clinically, the order of...",None
224001,Carvedilol,ANDA077614,6 ADVERSE REACTIONS Most common adverse events...,14 CLINICAL STUDIES 14.1 Heart Failure A total...,1 INDICATIONS AND USAGE Carvedilol tablets are...,4 CONTRAINDICATIONS Bronchial asthma or relate...,5 WARNINGS AND PRECAUTIONS Acute exacerbation ...,None,None,12.3 Pharmacokinetics Carvedilol tablets are r...,None,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,None,None,None,12.2 Pharmacodynamics Heart Failure The basis ...,None
224002,hydrocodone bitartrate and acetaminophen,ANDA207509,ADVERSE REACTIONS The following adverse reacti...,None,INDICATIONS AND USAGE Hydrocodone bitartrate a...,CONTRAINDICATIONS Hydrocodone Bitartrate and A...,None,"WARNINGS Addiction, Abuse, and Misuse Hydrocod...",PRECAUTIONS Risks of Driving and Operating Mac...,Pharmacokinetics The behavior of the individua...,None,CLINICAL PHARMACOLOGY Mechanism of Act

In [21]:
fda_df.to_excel('../data/openfda-04232024.xlsx',sheet_name='04232024')

### Scratch

In [ ]:
file_numbers = ['0001','0002','0003','0004','0005','0006','0007','0008','0009','0010','0011','0012']
fda_df = pd.DataFrame(columns=['id','adverse_reactions','indications_and_usage','contraindications','warnings_and_cautions','brand_name'])

for number in tqdm(file_numbers):
    f = open(f'../data/drug-label-{number}-of-0011.json','r')
    data = json.load(f)
    fda_df = extract_data(data,fda_df)



file_numbers = ['0001','0002','0003','0004','0005','0006','0007','0008','0009','0010','0011']
fda_df = pd.DataFrame(columns=['id','adverse_reactions','indications_and_usage','contraindications','warnings_and_cautions','brand_name'])

for number in tqdm(file_numbers):
    f = open(f'../data/drug-label-{number}-of-0011.json','r')
    data = json.load(f)
    fda_df = extract_data(data,fda_df)



In [ ]:
data['results'][0].keys()

# additional fields of interest potentially: nonclinical_toxicology, clinical pharmacology?

dict_keys(['spl_product_data_elements', 'indications_and_usage', 'dosage_and_administration', 'dosage_and_administration_table', 'dosage_forms_and_strengths', 'contraindications', 'warnings_and_cautions', 'adverse_reactions', 'drug_interactions', 'use_in_specific_populations', 'overdosage', 'description', 'clinical_pharmacology', 'clinical_pharmacology_table', 'nonclinical_toxicology', 'clinical_studies', 'references', 'how_supplied', 'information_for_patients', 'package_label_principal_display_panel', 'set_id', 'id', 'effective_time', 'version', 'openfda'])

In [ ]:
data['results'][0]['openfda'] # brand name, rxcui

{'application_number': ['ANDA065117'],
 'brand_name': ['AMOXICILLIN AND CLAVULANATE POTASSIUM'],
 'generic_name': ['AMOXICILLIN AND CLAVULANATE POTASSIUM'],
 'manufacturer_name': ['DIRECT RX'],
 'product_ndc': ['61919-019', '61919-401'],
 'product_type': ['HUMAN PRESCRIPTION DRUG'],
 'route': ['ORAL'],
 'substance_name': ['AMOXICILLIN', 'CLAVULANATE POTASSIUM'],
 'rxcui': ['308189', '617296'],
 'spl_id': ['b96b2e24-4192-31e2-e053-2a95a90a2356'],
 'spl_set_id': ['0173e9de-a995-4386-bb65-8fc2bbf347f9'],
 'package_ndc': ['61919-401-32', '61919-019-20'],
 'original_packager_product_ndc': ['65862-071', '66685-1002'],
 'unii': ['804826J2HU', 'Q42OMW3AT8']}